# [EX_04] 멋진 작사가 만들기

## 1. 데이터 다운로드
* Cloud Shell에서 다음 명령어 입력
 ```
 $ mkdir -p ~/aiffel/lyricist/models  #폴더를 생성
 $ ln -s ~/data ~/aiffel/lyricist/data  #폴더 연결
 ```

## 2. 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 3. 데이터 정제

In [2]:
import re
import numpy as np
import tensorflow as tf

In [3]:
def preprocess_sentence(sentence): 
    # 입력된 문장을
    sentence = sentence.lower().strip() #                     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #     2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'(" ")+', " ", sentence) #             3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z.!,¿]+", " ", sentence) #    4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() #                             5. 다시 양쪽 공백을 지웁니다
    sentence = "<start> " + sentence + " <end>" #             6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0 : continue # 문장의 길이가 0인 경우 건너뜀
    if sentence[-1] == ':': continue # 문장의 마지막이 ':'으로 끝날 경우 건너뜀 -> 화자가 표기된 의미없는 문장이기 때문
        
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue # 단어별로 토큰화 되기 때문에 15단어를 넘어가는 문장을 제외 -> 토큰 사이즈를 제한
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

## 4. 평가 데이터셋 분리

In [4]:
def tokenize(corpus):        

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 7000단어를 기억할 수 있는 tokenizer 생성 
        filters=' ',  # 문장 정제를 완료했기에 필터 필요치 않음
        oov_token="<unk>"  # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿈
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 15로 제한했기에 시쿼스 길이가 15 보다 긴 입력 데이터는 없음
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(
        tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print(tensor.shape)

[[   2   49    4 ...    0    0    0]
 [   2   15 2970 ...    0    0    0]
 [   2   33    7 ...    3    0    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  256  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f5264722760>
(156348, 15)


In [5]:
print(tensor[0]) # 잘 만들어졌는지 하나만 확인

[   2   49    4   94  300   60   52    9  946 6274    3    0    0    0
    0]


In [6]:
src_input = tensor[:, :-1] #마지막 토큰을 제외하고 학습 데이터(X)로 저장
tgt_input = tensor[:, 1:] #시작 토큰을 제외하고 타겟 데이터(y)으로 저장

In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=29)
# sklearn 라이브러리를 활용하여 데이터를 train과 validation으로 나눠줌

In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (125078, 14)
Target Train: (125078, 14)


In [10]:
# 모델을 피팅하는 과정에서 데이터를 numpy array 그대로 넣어주지 않고 데이터셋 객체를 만들어줄것
# tf.data.Dataset객체를 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의 기능이 제공된다고 함

BUFFER_SIZE = len(enc_train) # 전체 데이터수 보다 크거나 같은 값으로 설정해야 모두를 섞을 수 있음
BATCH_SIZE = 256 # 배치 사이즈 설정
# steps_per_epoch = len(enc_train) // BATCH_SIZE # 한 번의 epoch에 몇번의 스텝이 있는지 설정하는데 어디에 쓰이는지 모르겠음..(???)
                                               # 제외하겠음

# tokenizer에서 구축한 7000개 + 포함되지 않은 0:<pad> = 7001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듬
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# validation 데이터 또한 tf.data.Dataset객체로 만들어줌
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset.shuffle(len(enc_val))

## 5. 인공지능 만들기

In [11]:
# 모델을 만들어주는 클래스를 만듬
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # 임베딩 레이어는 입력 텐서의 단어 사전 인덱스 값을 이에 해당하는 워드 벡터로 바꿔주는 역할
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        # 자연어 처리에서 거의 기본적으로 쓰이는 RNN구조인 LSTMM 레이어를 두 층 추가해줌
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # 임베딩 레이어에 들어가는 인자
                     # 값이 클수록 단어의 추상적 의미를 잘 파악하지만 데이터가 충분하지 않으면 오히려 혼란의 야기함
hidden_size = 1024 # 모델을 처리하는 일꾼의 수라고 이해하면 쉬움
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [22]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[-1.95602930e-04,  3.38643120e-04, -2.79349741e-04, ...,
          9.39859892e-05,  1.01447076e-04, -9.90875924e-05],
        [-1.64626530e-04,  5.19412570e-04, -4.55870235e-04, ...,
          3.50399874e-04,  1.50611260e-04,  8.09396588e-05],
        [-1.06139814e-04,  8.37758707e-04, -1.76696849e-05, ...,
          3.54170159e-04,  4.37719864e-04,  2.02760086e-04],
        ...,
        [ 4.65008634e-04,  1.06632325e-03,  9.35234362e-04, ...,
         -6.32841256e-04, -7.50105362e-04,  2.04317446e-04],
        [ 3.50154267e-04,  1.56984001e-03,  1.27843372e-03, ...,
         -1.02228578e-03, -1.18253485e-03,  1.19892553e-04],
        [ 2.67095485e-04,  2.06813752e-03,  1.64669717e-03, ...,
         -1.43345783e-03, -1.57369801e-03,  6.01385291e-05]],

       [[-1.95602930e-04,  3.38643120e-04, -2.79349741e-04, ...,
          9.39859892e-05,  1.01447076e-04, -9.90875924e-05],
        [-1.85571960e-04,  8.10743601e-04, -3.

In [15]:
# 모델 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [66]:
print(tf.test.is_gpu_available()) # gpu가 사용가능한 상황인지 확인
tf.config.list_physical_devices('GPU') # 이렇게 바꿔서 쓰라고 추천함

True


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [30]:
optimizer = tf.keras.optimizers.Adam() # 옵티마이저는 Adam으로 설정
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer) # 모델 학습을 위한 값을 설정
# 우선 10번의 epoch 후에 val_loss가 2.2 아래로 떨어지는지 확인
history1 = model.fit(dataset,
                    epochs=10,
                   validation_data=dataset_val,
                   verbose=1)

Epoch 1/10
488/488 [==============================] - 93s 182ms/step - loss: 3.3715 - val_loss: 3.0065
Epoch 2/10
488/488 [==============================] - 89s 182ms/step - loss: 2.9119 - val_loss: 2.7915
Epoch 3/10
488/488 [==============================] - 89s 182ms/step - loss: 2.7473 - val_loss: 2.6456
Epoch 4/10
488/488 [==============================] - 89s 182ms/step - loss: 2.6201 - val_loss: 2.5254
Epoch 5/10
488/488 [==============================] - 89s 182ms/step - loss: 2.5103 - val_loss: 2.4137
Epoch 6/10
488/488 [==============================] - 89s 182ms/step - loss: 2.4107 - val_loss: 2.3162
Epoch 7/10
488/488 [==============================] - 89s 183ms/step - loss: 2.3184 - val_loss: 2.2245
Epoch 8/10
488/488 [==============================] - 89s 182ms/step - loss: 2.2319 - val_loss: 2.1402
Epoch 9/10
488/488 [==============================] - 89s 183ms/step - loss: 2.1491 - val_loss: 2.0554
Epoch 10/10
488/488 [==============================] - 89s 183ms/step - l

* 10 epochs안에 val_loss가 2.2 밑으로 떨어지는 것을 확인 -> val_loss가 1.9732로 나옴
* 더 높은 수준을 위해 예제에서처럼 20 epochs를 더 진행

In [31]:
history1 = model.fit(dataset,
                    epochs=20,
                   validation_data=dataset_val,
                   verbose=1)

Epoch 1/20
488/488 [==============================] - 90s 184ms/step - loss: 1.9900 - val_loss: 1.8954
Epoch 2/20
488/488 [==============================] - 89s 183ms/step - loss: 1.9126 - val_loss: 1.8166
Epoch 3/20
488/488 [==============================] - 89s 183ms/step - loss: 1.8363 - val_loss: 1.7409
Epoch 4/20
488/488 [==============================] - 89s 183ms/step - loss: 1.7615 - val_loss: 1.6641
Epoch 5/20
488/488 [==============================] - 89s 183ms/step - loss: 1.6884 - val_loss: 1.5924
Epoch 6/20
488/488 [==============================] - 89s 183ms/step - loss: 1.6172 - val_loss: 1.5243
Epoch 7/20
488/488 [==============================] - 89s 183ms/step - loss: 1.5493 - val_loss: 1.4569
Epoch 8/20
488/488 [==============================] - 89s 183ms/step - loss: 1.4849 - val_loss: 1.3940
Epoch 9/20
488/488 [==============================] - 89s 183ms/step - loss: 1.4245 - val_loss: 1.3377
Epoch 10/20
488/488 [==============================] - 90s 183ms/step - l

In [32]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence를 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듬
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

### 문장 만들어보기!!!

In [68]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love ma little nasty girl <end> '

#### 정상 작동 확인!!
"나의 작은 끔찍한 소녀를 사랑해"  
조금은 이상하지만 Lyric이라면 가능할법하다.

* 여러개의 문장을 만들어보자!

In [69]:
s1 = generate_text(model, tokenizer, init_sentence="<start> i m", max_len=20)
s2 = generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)
s3 = generate_text(model, tokenizer, init_sentence="<start> where is", max_len=20)
s4 = generate_text(model, tokenizer, init_sentence="<start> she s", max_len=20)
s5 = generate_text(model, tokenizer, init_sentence="<start> it is", max_len=20)
s6 = generate_text(model, tokenizer, init_sentence="<start> this is", max_len=20)

In [70]:
print(s1)
print(s2)
print(s3)
print(s4)
print(s5)
print(s6)

<start> i m gonna make it alright but not right now <end> 
<start> you are the one so i make sure i behave <end> 
<start> where is your heart , when i m not around <end> 
<start> she s got me runnin round and round <end> 
<start> it is a woken dream <end> 
<start> this is my legacy , legacy <end> 


>i m gonna make it alright but not right now 
you are the one so i make sure i behave  
where is your heart , when i m not around  
she s got me runnin round and round  
it is a woken dream  
this is my legacy , legacy  

* 이상하지만 용납 가능하다.  
이 정도면 시적이기도 하고..뭔가 반전이 있고 잘 이어지는 느낌이다.  
이건 다 legacy....

---
# 회고

(1) 텍스트 생성 모델은 처음으로 다루어 보았는데 모델 학습이 잘 되었다.  
문장을 만들어보니 아주 말이 안되는 결과는 나오지 않았다.  
그렇다고 아주 자연스러운 문장은 아니지만 조금 부정적인 뉘앙스가 더 많이 표현되는것 같다.

(2) 네트워크를 만들고 모델을 학습하기 전에 데이터를 전처리 하는 것이 상당히 중요하다는 것을 다시 한번 느낀다.  
그리고 그 과정 또한 간단하지 않고, 생각해야할 요소들이 많다.  
그럼에도 미리 해야할 것들을 적어놓고, 하나하나 작성을 해보는 방식으로 하니 큰 어려움은 없었다.  
다만 아직도 인덱스 관련하여 그 값이 포함되는지, 혹은 포함되지 않는지가 직관적으로 떠오르지 않는다.  

(3) lms 노드에서 말한대로 10 epochs가 지난 후에 val_loss가 대략 1.9 정도로 나오며 학습이 잘 되는 모습을 확인 할 수 있었다.  
총 30번의 epoch가 돌고 난 후에는 최종적으로 val_loss가 대략 0.97 정도 나왔는데 나쁘지 않은 값인것 같고,  
epoch 마다 계산된 val_loss를 봤을때 학습을 더 진행한다면 조금 더 낮아질 수도 있을 것 같다.

(4) 모델을 만들어주는 TextGenerator 클래스를 만드는 부분이 혼란스럽다.  
처음 model은 해당 클래스의 인스턴스로 선언이 되면서 아직 모델이 되었고, call 함수가 호출되어 모델이 빌드된것 같은데...  
call은 매직 메서드로 \_\_call\_\_ 이렇게 쓰여야 하는 것 아닌가..??  
그리고 model(src_sample) 이렇게 입력 샘플을 넣어줘서 모델을 호출해주어야만 네트워크가 완전히 생성된다.  
call이 매직 메서드로 쓰인것이 맞는지 더 알아보아야 할듯!!!! 

(5) 텍스트를 이용한 딥러닝은 아예 처음 접해보았다.  
결과를 보니 생각보다 흥미롭게 다가오지만, 역시나 익숙하지 않아서인지 어렵게 느껴진다.
어떤 분야를 선택하더라도 그 하나만 알고 나머지를 모르는 것은 매우 위험하기에,  
앞으로 텍스트를 다루지 않더라도 이런 기본적인 내용은 필히 숙지하고 있어야겠다.